In [6]:
from os import listdir
from os.path import isfile, join

statdir = "playerstats"
files = [join(statdir, f) for f in listdir(statdir) if isfile(join(statdir, f))]

In [7]:
import json

playerdata = []
for jf in files:
    with open(jf) as f:
        playerdata.append(json.load(f))

In [12]:
playerdata[0]

[{u'Assists': 10,
  u'Deaths': 9,
  u'GameDuration': 2355,
  u'GoldEarned': 14384,
  u'Kills': 2,
  u'Lane': u'TOP',
  u'MatchID': 2071436552,
  u'Role': u'SOLO',
  u'Winner': True},
 {u'Assists': 15,
  u'Deaths': 5,
  u'GameDuration': 1812,
  u'GoldEarned': 12123,
  u'Kills': 5,
  u'Lane': u'MIDDLE',
  u'MatchID': 2071524656,
  u'Role': u'SOLO',
  u'Winner': True},
 {u'Assists': 10,
  u'Deaths': 10,
  u'GameDuration': 2350,
  u'GoldEarned': 10598,
  u'Kills': 4,
  u'Lane': u'BOTTOM',
  u'MatchID': 2071527908,
  u'Role': u'DUO_SUPPORT',
  u'Winner': False},
 {u'Assists': 3,
  u'Deaths': 9,
  u'GameDuration': 1236,
  u'GoldEarned': 6681,
  u'Kills': 4,
  u'Lane': u'TOP',
  u'MatchID': 2071962569,
  u'Role': u'SOLO',
  u'Winner': False},
 {u'Assists': 3,
  u'Deaths': 6,
  u'GameDuration': 1703,
  u'GoldEarned': 7120,
  u'Kills': 1,
  u'Lane': u'JUNGLE',
  u'MatchID': 2072367463,
  u'Role': u'NONE',
  u'Winner': False},
 {u'Assists': 4,
  u'Deaths': 3,
  u'GameDuration': 1649,
  u'GoldEar

In [46]:
import numpy as np

ignorethese = ('MatchID', 'Lane', 'Role')

def filterdata(data, lane, role):
    return filter(lambda stat: (lane == "ANY" or stat["Lane"] == lane) and 
                                    (role == "ANY" or stat["Role"] == role), 
                                   data)

def createAvgStats(stats, lane, role):
    avgstats = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in avgstats:
                    avgstats[key] += stat[key]
                else:
                    avgstats[key] = stat[key]
    for key in avgstats:
        avgstats[key] /= float(len(filteredstats))
    avgstats["Lane"] = lane
    avgstats["Role"] = role
    return avgstats

def createPlayersAvgStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    return createAvgStats(arr, lane, role)

def createStatsDistribution(stats, lane, role):
    statdistribution = {}
    filteredstats = filterdata(stats, lane, role)
    if len(filteredstats) == 0:
        return None
    for stat in filteredstats:
        for key in stat:
            if key not in ignorethese:
                if key in statdistribution:
                    statdistribution[key].append(stat[key])
                else:
                    statdistribution[key] = [stat[key]]
        
    statdistribution["Lane"] = lane
    statdistribution["Role"] = role
    return statdistribution

def createStdStats(stats, lane, role):
    sd = createStatsDistribution(stats, lane, role)
    
    if sd == None:
        return None
    
    stds = {}
    for key in sd:
        if key == "Lane" or key == "Role":
            stds[key] = sd[key]
        else:
            stds[key] = np.std(sd[key])
    
    return stds

def createPlayersStdStats(playerdata, lane, role):
    arr = []
    for p in playerdata:
        stdp = createStdStats(p, lane, role)
        ap = createAvgStats(p, lane, role)
        if ap:
            ap["GamesPlayed"] = len(filterdata(p, lane, role))
            arr.append(ap)
    
    return createStdStats(arr, lane, role)

In [47]:
avgstatdata = []
avgstatdata.append(createPlayersAvgStats(playerdata, "TOP", "SOLO"))
avgstatdata.append(createPlayersAvgStats(playerdata, "JUNGLE", "NONE"))
avgstatdata.append(createPlayersAvgStats(playerdata, "MIDDLE", "SOLO"))
avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_CARRY"))
avgstatdata.append(createPlayersAvgStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
avgstatdata.append(createPlayersAvgStats(playerdata, "ANY", "ANY"))

In [40]:
with open('player_stats_mean.json', 'w') as f:
    json.dump(avgstatdata, f)

In [48]:
stddata = []
stddata.append(createPlayersStdStats(playerdata, "TOP", "SOLO"))
stddata.append(createPlayersStdStats(playerdata, "JUNGLE", "NONE"))
stddata.append(createPlayersStdStats(playerdata, "MIDDLE", "SOLO"))
stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_CARRY"))
stddata.append(createPlayersStdStats(playerdata, "BOTTOM", "DUO_SUPPORT"))
stddata.append(createPlayersStdStats(playerdata, "ANY", "ANY"))

In [42]:
with open('player_stats_std.json', 'w') as f:
    json.dump(stddata, f)

In [49]:
avgstatdata[0]

{u'Assists': 6.629942137586515,
 u'Deaths': 5.173745185352843,
 u'GameDuration': 1766.8624213622786,
 'GamesPlayed': 22.250599520383695,
 u'GoldEarned': 11602.671462725126,
 u'Kills': 5.128346953476573,
 'Lane': 'TOP',
 'Role': 'SOLO',
 u'Winner': 0.5082603561487581}

In [50]:
stddata[0]

{u'Assists': 2.6241522565323234,
 u'Deaths': 1.6429886824905142,
 u'GameDuration': 217.21645661682808,
 'GamesPlayed': 34.771895000053874,
 u'GoldEarned': 1968.4573198300452,
 u'Kills': 2.3144693087731061,
 'Lane': 'TOP',
 'Role': 'SOLO',
 u'Winner': 0.27235967985510251}

In [118]:
distdata = createStatsDistribution(stats, "ANY", "ANY")